In [13]:
from funcx.sdk.client import FuncXClient
import time


fxc = FuncXClient(force_login=True)
fxc.throttling_enabled = False
print(fxc.base_url, fxc.throttling_enabled)

Please paste the following URL in a browser:
https://auth.globus.org/v2/oauth2/authorize?client_id=4cf29807-cf21-49ec-9443-ff9a3fb9f81c&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=https%3A%2F%2Fauth.globus.org%2Fscopes%2Ffacd7ccc-c5f4-42aa-916b-a0e270e2c2a9%2Fall&state=_default&response_type=code&code_challenge=Tdl37KdO10XbO0a6oI7zE9hKFVSmX7E059w2vM4WYRg&code_challenge_method=S256&access_type=offline&prefill_named_grant=FuncX+SDK+Login
Please Paste your Auth Code Below: 
Q3dlGB1T1VtTTD6xofLCr68ulUmq5e
https://funcx.org/api/v1 False


In [14]:
endpoints = ['709118de-1103-463f-8425-281eb93b55ff',   #Theta covid19
             '67e95158-8bda-4b1f-a0ef-31a1626eba00',   #Ryan ep
            ]

In [15]:
def get_endpoints_status(endpoints):
    endpoint_status = {}
    for ep in endpoints:
        endpoint_status[ep] = fxc.get_endpoint_status(ep)
    return endpoint_status

In [16]:
get_endpoints_status(endpoints)

{'709118de-1103-463f-8425-281eb93b55ff': [{'active_managers': 49,
   'cores_per_worker': 1.0,
   'idle_workers': 49,
   'managers': 49,
   'max_blocks': 1,
   'max_workers_per_node': 1,
   'mem_per_worker': None,
   'min_blocks': 1,
   'new_core_hrs': 661.5935779148131,
   'nodes_per_block': 1,
   'outstanding_tasks': {'RAW': 0},
   'pending_tasks': 0,
   'prefetch_capacity': 10,
   'scaling_enabled': True,
   'scheduler_mode': 'soft',
   'total_core_hrs': 448691.67,
   'total_cores': 1568,
   'total_mem': 10648.9,
   'total_workers': 49,
   'worker_mode': 'no_container'},
  {'active_managers': 49,
   'cores_per_worker': 1.0,
   'idle_workers': 49,
   'managers': 49,
   'max_blocks': 1,
   'max_workers_per_node': 1,
   'mem_per_worker': None,
   'min_blocks': 1,
   'new_core_hrs': 664.9825225365348,
   'nodes_per_block': 1,
   'outstanding_tasks': {'RAW': 0},
   'pending_tasks': 0,
   'prefetch_capacity': 10,
   'scaling_enabled': True,
   'scheduler_mode': 'soft',
   'total_core_hrs':